<a href="https://colab.research.google.com/github/nforesperance/TensorFlow/blob/master/Automatic_Text_Generation_Using_Keras_and_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests
import os

TensorFlow 2.x selected.


In [0]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
from google.colab import drive
drive.mount('/content/drive')
checkpoint_path = "auto/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
save_path = "/content/drive/My Drive/Colab Notebooks/models/auto_text.h5"

In [0]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')
response.text[:10]
data = response.text.split('\n')
data = data[253:]
data = " ".join(data)

In [0]:
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens
tokens = clean_text(data)
# print(tokens[:50])

In [0]:
length = 10 + 1 # 2 words will be used to predict 1 word
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000:
    break

print(len(lines))

The tokenizer converts to digits the then put them back in the same order as lines

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

vocab_size = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

## LSTM Model

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [0]:
model = tf.keras.models.load_model(save_path)

In [0]:
for _ in range(20):
  del model
  model = tf.keras.models.load_model(save_path)
  model.fit(X, y, batch_size = 250, epochs = 20)
  model.save(save_path)


In [0]:
model.save(save_path)

In [0]:
del model
model = tf.keras.models.load_model(save_path)

In [0]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre') # extracting last text_seq_lenght words

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [0]:
seed_text = lines[102]

In [0]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 50)